# Training Process
This run has an example of running a training job

1. Creating AI template and instance

In [1]:
import os.path
import shutil

from superai.meta_ai.ai import AITemplate, AI, TrainingOrchestrator
from superai.meta_ai.parameters import Config
from superai.meta_ai.schema import Schema

In [2]:
if os.path.exists(".AISave"):
    shutil.rmtree(".AISave")

In [3]:
template = AITemplate(
    input_schema=Schema(),
    output_schema=Schema(),
    configuration=Config(),
    name="MnistTrainingTemplate",
    description="Template of Sample MNIST training",
    model_class="MnistModel",
    requirements=["tensorflow-gpu", "polyaxon"],
    artifacts={"run": "resources/runDir/run_this.sh"},
    code_path=["resources/runDir"],
)
ai = AI(
    ai_template=template,
    input_params=template.input_schema.parameters(),
    output_params=template.output_schema.parameters(),
    name="mnist_training",
    version=1,
    description="AI instance of sample MNIST training",
)

[02/18/22 12:28:22] INFO     Copying all code_path content - MainThread             ]8;id=1645183702.260383-349406;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:264

                    INFO     Saved model in .AISave/mnist_training/1 - MainThread   ]8;id=1645183702.2701209-733298;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:833

## Create and push training container
Use the following interface to create and push the training container.

> Note: Later, this interface will contain implementation to connect to meta-ai to create a training job.

In [4]:
ai._id = "DAAD6583-1FD7-4718-8B12-333701894FDB".lower()

ai.training_deploy(orchestrator=TrainingOrchestrator.AWS_EKS, skip_build=False, enable_cuda=True)

[02/18/22 12:28:24] INFO     Base image                                            ]8;id=1645183704.767591-911529;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1208
                             'superai-model-s2i-python3711-gpu-seldon:1' found               
                             locally. - MainThread

                    INFO     No change in pip layer. Reusing old layers from image ]8;id=1645183704.776865-547354;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1223
                             sha256:dd3575c4886c6ffff43846efef834b1b4ac955d6941ee6           
                             5aa044db256e71abf3... - MainThread

                    INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1645183704.78106-476076;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1303
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True . mnist_training-pip-layer:1                 
                             mnist_training:1' - MainThread

[02/18/22 12:28:25] INFO     Built main container `mnist_training:1` - MainThread  ]8;id=1645183705.9484632-91013;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1245

                    INFO     Time taken to build: 1.21s - MainThread               ]8;id=1645183705.951343-570269;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1246

...

[02/18/22 12:28:27] INFO     Logging in to ECR... - MainThread                ]8;id=1645183707.434641-493726;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:246

[02/18/22 12:28:30] INFO     Tagging to `185169359328.dkr.ecr.us-east-1.amazo ]8;id=1645183710.224075-238165;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:249
                             naws.com/models/dev/daad6583-1fd7-4718-8b12-3337                
                             01894fdb/mnist_training:1` - MainThread

                    INFO     Pushing image... - MainThread                    ]8;id=1645183710.240448-601983;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:252

Output()

[02/18/22 12:28:36] INFO      Image pushed successfully to 185169359328.dkr.e ]8;id=1645183716.863683-309279;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:291
                             cr.us-east-1.amazonaws.com/models/dev/daad6583-1                
                             fd7-4718-8b12-333701894fdb/mnist_training:1  -                  
                             MainThread

# Starting training manually
To start training manually, in a separate terminal run, `polyaxon port-forward` to connect to Polyaxon API.

Then run the following cell
> Note: triggering training will be done from meta-ai in the future.

In [5]:
!cat polyaxonfile.yaml

version: 1.1
kind: component
tags: [examples, keras]

inputs:
- {name: conv1_size, type: int, value: 32, isOptional: true}
- {name: conv2_size, type: int, value: 64, isOptional: true}
- {name: dropout, type: float, value: 0.8, isOptional: true}
- {name: hidden1_size, type: int, value: 500, isOptional: true}
- {name: optimizer, type: str, value: adam, isOptional: true}
- {name: log_learning_rate, type: int, value: -3, isOptional: true}
- {name: epochs, type: int, value: 10, isOptional: true}

run:
  kind: job
  container:
    image: 185169359328.dkr.ecr.us-east-1.amazonaws.com/models/dev/daad6583-1fd7-4718-8b12-333701894fdb/mnist_training:1
#    workingDir: "{{ globals.artifacts_path }}/polyaxon-examples/in_cluster/keras/mnist"
    command: ["/opt/conda/envs/env/bin/superai", "ai", "train"]
    imagePullPolicy: Always
    args: ["-p", "/home/model-server/",
           "-tp","/tmp",
           "-mp","/tmp",
           "-m","conv1_size={{ conv1_size }}",
           "-m","conv2_size={{ con

In [ ]:
!polyaxon run -f polyaxonfile.yaml -u -l

Creating a new run...
A new run `aa3d02416d554149b3620083348ddd32` was created
You can view this run on Polyaxon UI: http://localhost:8000/ui/default/dev/runs/aa3d02416d554149b3620083348ddd32/
[================================] 443605/443605 - 00:00:00
Artifacts uploaded
Run is approved
Starting logs for run: <Name: None> - <uuid: aa3d02416d554149b3620083348ddd32>
warning
running
2022-02-18 12:34:38.935113+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 12:34:38.935136+01:00 | Available envs:
2022-02-18 12:34:38.935139+01:00 | - local
2022-02-18 12:34:38.935142+01:00 | - dev
2022-02-18 12:34:38.935145+01:00 | - sandbox
2022-02-18 12:34:38.935148+01:00 | - stg
2022-02-18 12:34:38.935151+01:00 | - prod
2022-02-18 12:34:38.935153+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 12:34:38.935155+01:00 | [02/18/22 11:34:38] INFO     Setting config : prod - MainThread    